<a href="https://colab.research.google.com/github/seunghyeokkim/ByeolDaJul/blob/main/kobert_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split


In [8]:
# 필요한 라이브러리들을 임포트합니다.
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

# 데이터를 불러오는 함수를 정의합니다.
def load_and_parse_data(filepath):
    data = {
        "label": [],
        "intent_pair1": [],
        "intent_pair2": []
    }

    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            parts = line.strip().split("\t")
            data["label"].append(int(parts[0]))
            data["intent_pair1"].append(parts[1])
            data["intent_pair2"].append(parts[2])

    return pd.DataFrame(data)

# 위에서 정의한 함수를 사용하여 데이터를 불러옵니다.
df = load_and_parse_data('/content/sae4k_v2.txt')

# KcBERT의 토크나이저와 모델을 불러옵니다.
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())

# 데이터셋 클래스를 정의합니다. 이 클래스는 토큰화와 인코딩을 수행합니다.
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}

# 데이터셋과 DataLoader를 준비합니다.
texts = df['intent_pair1'].tolist()
labels = df['label'].tolist()
max_length = 128
dataset = CustomDataset(texts, labels, tokenizer, max_length)

# 데이터셋을 훈련용과 검증용으로 분리합니다.
train_dataset, valid_dataset = train_test_split(dataset, test_size=0.15, random_state=42)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#모델을 훈련하고 검증하는 코드가 들어갑니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
learning_rate = 1e-5
epochs = 5
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
batch_size = 16


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
from utils import compute_accuracy
def compute_accuracy(model, data_loader, device):
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0
    total = 0

    with torch.no_grad():  # 그래디언트 계산을 비활성화
        for data in data_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100.0 * correct / total
    return accuracy


ModuleNotFoundError: No module named 'utils'

In [18]:
from torch.utils.data import random_split, DataLoader

# 원하는 데이터의 일부 비율 설정 (예: 1%)
fraction = 0.01
num_samples = int(fraction * len(valid_dataloader.dataset))

# 데이터셋에서 일부만 샘플링
partial_valid_dataset, _ = random_split(valid_dataloader.dataset, [num_samples, len(valid_dataloader.dataset) - num_samples])

# 새로운 DataLoader 생성
partial_valid_dataloader = DataLoader(partial_valid_dataset, batch_size=batch_size, shuffle=True)

accuracy = compute_accuracy(model, partial_valid_dataloader, device)
print(f"Partial Validation Accuracy: {accuracy:.4f}")

ModuleNotFoundError: No module named 'utils'

In [20]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
import torch

# 데이터 불러오기 및 파싱 함수 정의
def load_and_parse_data(filepath):
    data = {"label": [], "intent_pair1": [], "intent_pair2": []}

    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            parts = line.strip().split("\t")
            data["label"].append(int(parts[0]))
            data["intent_pair1"].append(parts[1])
            data["intent_pair2"].append(parts[2])

    return pd.DataFrame(data)

# 데이터 불러오기
filepath = '/content/sae4k_v2.txt'
df = load_and_parse_data(filepath)

# 데이터 샘플링 (원본의 5% 사용)
sample_frac = 0.05
df = df.sample(frac=sample_frac, random_state=42)

# KcBERT 토크나이저 및 모델 초기화
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base', num_labels=df['label'].nunique())

# CUDA 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 데이터 토큰화 및 DataLoader 생성
def tokenize_data(df, tokenizer):
    sentences = df['intent_pair1'].values
    labels = df['label'].values

    input_ids = []
    attention_masks = []

    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=64,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

input_ids, attention_masks, labels = tokenize_data(df, tokenizer)

# 데이터셋 분할
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.1)

# DataLoader 설정
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 1

# 모델 학습
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}")

    for step, batch in enumerate(progress):
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss

        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Training loss: {avg_train_loss:.2f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th

KeyboardInterrupt: 

In [11]:
def valid_label(label):
    labels = [
        "0 (예/아니오)",
        "1 (대체적인)",
        "2 (wh- 질문)",
        "3 (금지)",
        "4 (요구 사항)",
        "5 (강력한 요구 사항)"
    ]
    return labels[label]

# 테스트하고자 하는 텍스트 데이터
input_data = [
    "나 물좀 가져다줘"
]

input_encodings = tokenizer(input_data, padding=True, truncation=True, return_tensors="pt").to(device)

# 모델에 입력 데이터 전달
with torch.no_grad():
    output = model(**input_encodings)

# 예측 결과 확인
logits = output.logits
predicted_labels = logits.argmax(dim=1)

# 예측 결과 출력
for i, input_text in enumerate(input_data):
    predicted_label = predicted_labels[i].item()
    print(f"Input: {input_text} - Predicted Label: {valid_label(predicted_label)}")


Input: 나 물좀 가져다줘 - Predicted Label: 5 (강력한 요구 사항)


In [ ]:
# 모델 저장
model_save_path = "kc_bert_classifier.pth"
torch.save(model.state_dict(), model_save_path)

# 모델 로드
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.eval()

kcbert

In [12]:
!pip install soynlp emoji
!pip install torch torchvision
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 4.1 MB/s eta 0:00:00


In [13]:
!pip install transformers --upgrade


In [14]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)

    return x

In [15]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
import torch
# 데이터 불러오기 함수
def load_and_parse_data(filepath):
    data = {
        "label": [],
        "intent_pair1": [],
        "intent_pair2": []
    }

    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            parts = line.strip().split("\t")
            data["label"].append(int(parts[0]))
            data["intent_pair1"].append(parts[1])
            data["intent_pair2"].append(parts[2])

    return pd.DataFrame(data)

# 데이터 불러오기
df = load_and_parse_data('/content/sae4k_v2.txt')

# 원래 데이터셋의 5%만 사용
sample_frac = 0.05
df = df.sample(frac=sample_frac, random_state=42)

# KcBERT 토크나이저 및 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
model = BertForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=df['label'].nunique())

# CUDA 설정 (GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 데이터 토큰화
# 2. 데이터 토큰화 및 DataLoader 생성
sentences = df['intent_pair1'].values
labels = df['label'].values

input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                        truncation=True
                   )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.1)

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 모델 학습 설정 및 학습
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 1

for epoch in range(epochs):
    model.train()

    total_loss = 0
    progress = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}")

    for step, batch in enumerate(progress):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Training loss: {avg_train_loss:.2f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th

KeyboardInterrupt: 

평가

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten() #flat_accuracy는 예측된 라벨과 실제 라벨을 비교하여 정확도를 계산하는 함수
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [ ]:
from tqdm import tqdm

# 평가 모드
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 검증 데이터셋에 대해
for batch in tqdm(validation_dataloader, desc="Evaluating"):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # 그래디언트 계산 X
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]

    # 정확도 계산
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))


In [ ]:
# 1. 모델의 가중치 얻기
weights = model.state_dict()
num_weights = len(weights)

# 2. 검증 데이터셋에 대한 정확도 계산

# 평가 모드
model.eval()

# 변수 초기화
eval_accuracy = 0
nb_eval_steps = 0

# 검증 데이터셋에 대해
for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # 그래디언트 계산 X
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]

    # 정확도 계산
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

accuracy = eval_accuracy/nb_eval_steps

print(f"Number of weights (parameter groups) in the model: {num_weights}")
print(f"Validation Accuracy: {accuracy:.2f}")


In [ ]:
weights = model.state_dict()

total_params = 0

for key, value in weights.items():
    num_params = torch.numel(value)
    total_params += num_params
    print(f"{key} contains {num_params} parameters")

print(f"Total number of parameters in the model: {total_params}")


In [ ]:
# 평가 모드
model.eval()

# 라벨 5의 빈도를 저장할 변수
label_5_count = 0

# 검증 데이터셋에 대한 예측 수행
for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # 그래디언트 계산 X
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]

    # 예측된 라벨
    logits = logits.detach().cpu().numpy()
    predictions = np.argmax(logits, axis=1)

    # 라벨 5의 빈도 계산
    label_5_count += np.sum(predictions == 5)

print(f"Label 5 appears {label_5_count} times in the predictions.")


휴먼평가

In [ ]:
import numpy as np

def flat_accuracy(preds, labels): #flat_accuracy는 예측된 라벨과 실제 라벨을 비교하여 정확도를 계산하는 함수
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [ ]:
def predict(text, model, tokenizer):
    model.eval()

    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=64,
        pad_to_max_length=True,
        return_token_type_ids=True,
        truncation=True
    )

    input_ids = torch.tensor([inputs["input_ids"]]).to(device)
    attention_mask = torch.tensor([inputs["attention_mask"]]).to(device)

    with torch.no_grad():
        logits = model(input_ids, token_type_ids=None, attention_mask=attention_mask)[0]

    logits = logits.detach().cpu().numpy()
    prediction = np.argmax(logits, axis=1)

    return prediction[0]


In [ ]:
text = " 관악산 등반할 때 일반 운동화가 아닌 등산화를 신어야해"
predicted_label = predict(text, model, tokenizer)
print(f"Predicted label for text '{text}': {predicted_label}")


In [ ]:
# 훈련 데이터에서 5번 라벨로 분류된 텍스트 샘플 추출
label_5_texts = df[df['label'] == 5]['intent_pair1'].tolist()

# 5번 라벨로 분류된 텍스트로 예측 수행
for text in label_5_texts:
    predicted_label = predict(text, model, tokenizer)
    print(f"Text: {text} -> Predicted Label: {predicted_label}")


In [ ]:
!pip install torchviz


In [ ]:
import torch
from torchviz import make_dot

# Assuming 'tokenizer' is defined and has a method `model_max_length`
# Create a tensor filled with a repeated pattern of indices, and ensure it's of type LongTensor.
dummy_input = torch.randint(0, tokenizer.vocab_size, (1, tokenizer.model_max_length)).to(device).long()

# Forward pass
outputs = model(dummy_input)
logits = outputs[0]

# Visualization
dot = make_dot(logits, params=dict(model.named_parameters()))
dot.format = 'png'
dot.render("model_graph")


In [ ]:
#모델 가중치 저장
model_save_path = "/content/kcbert_finetuned_model.pth"
torch.save(model.state_dict(), model_save_path)


In [26]:
# 모델 인스턴스 생성
model = BertForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=df['label'].nunique())
model.to(device)

# 저장된 가중치 로드
model.load_state_dict(torch.load(model_save_path))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'model_save_path' is not defined